In [18]:
import pandas as pd
import xgboost as xgb
import os

####### read files: #########
def readDF(fileDir):
    featureDF = pd.read_csv(fileDir)
    return featureDF

### descrição do problema:

### Ja ta com as features
### Ja ta tudo na mesma tabela
### Problema de classificacao

###

fileDir = os.path.join(os.getcwd(), "data/tabular-playground-series-may-2022", "train.csv")

featureDF = readDF(fileDir)
featureDF.set_index('id')

catFList = ['f_27']
discFList = ["f_07","f_08","f_09","f_10","f_11","f_12","f_13","f_14","f_15","f_16","f_17","f_18","f_29","f_30"]
tgtFList = ["target"]
numFList = list(set(featureDF.columns) - set(tgtFList) - set(discFList) - set(catFList))

In [16]:

z = x + y

print(z)

ValueError: Unable to coerce to Series, length must be 32: given 3